## 14.1 stdout으로 보낸 결과물 테스팅

In [1]:
# mymodule.py

def urlprint(protocol, host, domain):
    url = '{}://{}.{}'.format(protocol, host, domain)
    print(url) # -> sys.stdout으로 보냄

In [4]:
from io import StringIO
from unittest import TestCase
from unittest.mock import patch

## unittest.TestCase에서 test_ 로 시작하면 모두 테스트 메소드가 됨
class TestURLPrint(TestCase):
    def test_url_gets_to_stdout(self):
        protocol = 'http'
        host = 'www'
        domain = 'example.com'
        expected_url = '{}://{}.{}\n'.format(protocol, host, domain)
        
        with patch('sys.stdout', new=StringIO()) as fake_out:
            urlprint(protocol, host, domain)
            self.assertEqual(fake_out.getvalue(), expected_url)

## 14.5 테스트 실패 예측과 건너뜀

In [5]:
import unittest
import os
import platform

class Tests(unittest.TestCase):
    def test_0(self):
        self.assertTrue(True)
        
    @unittest.skip('skipped test')
    def test_1(self):
        self.fail('should have failed!')
        
    @unittest.skipIf(os.name == 'posix', 'Not supported on Unix')
    def test_2(self):
        import winreg
        
    @unittest.skipUnless(platform.system() == 'Darwin', 'Mac specific test')
    def test_3(self):
        self.assertTrue(True)
        
    @unittest.expectedFailure  #테스트가 실패하면 성공으로 간주, 테스트에 통과하면 실패로 간주
    def test_4(self):
        self.assertEqual(2+2, 5)
    

* skipIf(), skipUnless() : 특정 플랫폼이나 파이썬 버전의 조건 만족할때만 적용시킬때좋음

In [7]:
if __name__ == '__main__':
    unittest.main()

E
ERROR: /run/user/1000/jupyter/kernel-87d75c30-c0db-4662-85ea-298a713c434d (unittest.loader._FailedTest)
----------------------------------------------------------------------
AttributeError: module '__main__' has no attribute '/run/user/1000/jupyter/kernel-87d75c30-c0db-4662-85ea-298a713c434d'

----------------------------------------------------------------------
Ran 1 test in 0.001s

FAILED (errors=1)


SystemExit: True

/home/jiyeong/.local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:3299: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## 14.9 다른 예외에 대한 응답으로 예외 발생

In [13]:
def example():
    try:
        int ('N/A')
    except ValueError as e:
        raise RuntimeError('A parsing error occurred') from e

In [14]:
example()

RuntimeError: A parsing error occurred

In [15]:
try:
    example()
except RuntimeError as e:
    print ("It didn't work:", e)
    if e.__cause__:
        print ('Cause:', e.__cause__)

It didn't work: A parsing error occurred
Cause: invalid literal for int() with base 10: 'N/A'


In [16]:
def example2():
    try:
        int ('N/A')
    except ValueError:
        raise RuntimeError('A parsing error occured') from None

In [17]:
example2()

RuntimeError: A parsing error occured

## 14.13 프로파일링과 타이밍

* decorator로 만들기

In [20]:
import time
from functools import wraps

def timethis(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        r = func(*args, **kwargs)
        end = time.perf_counter()
        print('{}.{} : {}'.format(func.__module__, func.__name__, end-start))
    return wrapper

In [21]:
@timethis
def countdown(n):
    while n > 0 :
        n -= 1
        
countdown(10000000)

__main__.countdown : 0.4176428880000458


* contextmanager

In [22]:
from contextlib import contextmanager

@contextmanager
def timeblock(label):
    start = time.perf_counter()
    try:
        yield
    finally:
        end = time.perf_counter()
        print ('{}:{}'.format(label, end-start))

In [23]:
with timeblock('counting'):
    n = 10000000
    while n > 0 :
        n-=1
    

counting:0.665010410000832


* 특정 코드만 

In [24]:
from timeit import timeit
timeit('math.sqrt(2)', 'import math')

0.09708353400128544

In [25]:
timeit('sqrt(2)', 'from math import sqrt')

0.07902823099902889